## 머신러닝 기법으로 스팸메일 분류
* Spam_전처리완료.csv 파일 이용
- MachineLearning 방법으로 분류해서 정확도
    - CountVectorizer + NaiveBayes
    - TfidVectorizer + LogisticRegression



In [82]:
from google.colab import files
up = files.upload()
import warnings
warnings.filterwarnings('ignore')

MessageError: ignored

In [ ]:
df = pd.read_csv('spam_전처리완료.csv', encoding='latin1')
df.head(3)

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...


In [ ]:
x = df.v2
y = df.v1
x.shape, y.shape

((5169,), (5169,))

In [ ]:
df.v2 = df.v2.str.replace('[^A-Za-z0-9 ')

In [ ]:
# 구둣점 제거, 소문자 변환
import re

def preprocessing(s):
    s = s.encode('utf8').decode('ascii','ignore')
    s = re.sub('[^a-z0-9 ]', '', s.lower())
    return s

In [ ]:
X_data = [preprocessing(sent) for sent in x]
X_data[2]

'free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s'

In [ ]:
import numpy as np
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_data, y, stratify=y, test_size = 0.2, random_state=2022 
)
y_train.value_counts()

0    3613
1     522
Name: v1, dtype: int64

In [83]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

In [84]:
pipeline1 = Pipeline([ 
    ('cvect', CountVectorizer(stop_words='english')),
    ('nb', MultinomialNB())
])
params = {
    'cvect__max_df': [0.9, 0.95],
    'cvect__ngram_range': [(1,1),(1,2)]
}

In [85]:
grid_pipe1 = GridSearchCV(
    pipeline1, params, scoring='accuracy', cv=3, n_jobs=-1
)
grid_pipe1.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('cvect',
                                        CountVectorizer(stop_words='english')),
                                       ('nb', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'cvect__max_df': [0.9, 0.95],
                         'cvect__ngram_range': [(1, 1), (1, 2)]},
             scoring='accuracy')

In [86]:
grid_pipe1.best_params_ 

{'cvect__max_df': 0.9, 'cvect__ngram_range': (1, 2)}

In [87]:
grid_pipe1.best_estimator_.score(X_test, y_test)

0.9816247582205029

## 2) TfidfVectorizer + LogisticRegression

In [88]:
pipeline2 = Pipeline([ 
    ('tvect', TfidfVectorizer(stop_words='english')),
    ('lr', LogisticRegression(random_state=2022))
])
params = {
    'tvect__max_df': [0.9, 0.95],
    'tvect__ngram_range': [(1,1),(1,2)],
    'lr__C': [1,10]
}

In [89]:
grid_pipe2 = GridSearchCV(
    pipeline2, params, scoring='accuracy', cv=3, n_jobs=-1
)
%time grid_pipe2.fit(X_train, y_train)

CPU times: user 1.18 s, sys: 12.6 ms, total: 1.19 s
Wall time: 5.4 s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tvect',
                                        TfidfVectorizer(stop_words='english')),
                                       ('lr',
                                        LogisticRegression(random_state=2022))]),
             n_jobs=-1,
             param_grid={'lr__C': [1, 10], 'tvect__max_df': [0.9, 0.95],
                         'tvect__ngram_range': [(1, 1), (1, 2)]},
             scoring='accuracy')

In [91]:
grid_pipe2.best_estimator_.score(X_test, y_test)

0.9700193423597679